In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical 
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten

# Load the dataset
data = pd.read_csv('english.csv').astype('float32')
dataframe = pd.DataFrame(data)

# Extract features (x) and labels (y)
x = dataframe.drop('0', axis=1)
y = dataframe['0']

# Normalize pixel values
x = x / 255.0
y = y / 255.0

# Reshape the features for CNN input
x = x.values.reshape(-1, 28, 28, 1)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# One-hot encode the labels
y_train = to_categorical(y_train, num_classes=26)
y_test = to_categorical(y_test, num_classes=26)

# Build the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(26, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, epochs=3, validation_data=(x_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Get predictions
y_pred = model.predict(x_test)

# Convert predictions and true labels to character indices
y_pred_indices = np.argmax(y_pred, axis=1)
y_test_indices = np.argmax(y_test, axis=1)

# Create confusion matrix
conf_mat = confusion_matrix(y_test_indices, y_pred_indices)

# Display the confusion matrix using a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=range(26), yticklabels=range(26))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Display classification report
print(classification_report(y_test_indices, y_pred_indices))

# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy Over Epochs')
plt.legend()
plt.show()


# Save the trained model
model.save('handwritten_model.h5')

# Load the trained model
from keras.models import load_model
loaded_model = load_model('handwritten_model.h5')

# GUI Setup
from tkinter import *
from PIL import Image, ImageDraw

# Function to preprocess and predict the drawn image
# Update the predict_image function
def predict_image():
    # Get the drawn image from the canvas
    img = cv2.resize(cv2.cvtColor(np.array(canvas_image), cv2.COLOR_BGR2GRAY), (28, 28))
    
    # Normalize pixel values
    img = img / 255.0
    
    # Reshape for model input
    img = img.reshape(1, 28, 28, 1)

    # Make prediction using the loaded model
    probabilities = loaded_model.predict(img)
    
    # Get the predicted class (index with the highest probability)
    predicted_class = np.argmax(probabilities)
    
    # Display the prediction
    result_label.config(text=f"Prediction: {chr(predicted_class + 65)}")


# Function to clear the canvas
def clear_canvas():
    canvas.delete("all")
    result_label.config(text="Prediction: ")

# GUI Setup
root = Tk()
root.title("Handwritten Character Recognition")

# Create a canvas for drawing
canvas = Canvas(root, width=200, height=200, bg="white")
canvas.pack()

# Create an image for drawing
canvas_image = Image.new("RGB", (200, 200), "white")
draw = ImageDraw.Draw(canvas_image)

# Buttons
predict_button = Button(root, text="Predict", command=predict_image)
predict_button.pack()

clear_button = Button(root, text="Clear", command=clear_canvas)
clear_button.pack()

# Label for prediction result
result_label = Label(root, text="Prediction: ")
result_label.pack()

# Function to handle mouse events for drawing
def paint(event):
    x1, y1 = (event.x - 1), (event.y - 1)
    x2, y2 = (event.x + 1), (event.y + 1)
    canvas.create_oval(x1, y1, x2, y2, fill="black", width=5)
    draw.line([x1, y1, x2, y2], fill="black", width=5)

canvas.bind("<B1-Motion>", paint)

root.mainloop()


ValueError: could not convert string to float: 'Img/img001-001.png'